In [1]:
import os
import yaml
import openai

from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter


In [2]:
with open('../api.yaml', 'r') as f:
    api = yaml.safe_load(f)
openai.api_key = api['openai']
os.environ['OPENAI_API_KEY'] = api['openai']

In [3]:
# with open('./text_files_rag/hiraya_whisper_large.txt', 'r') as f:
with open('../text_files/softbank_world_2023_whisper_large.txt', 'r') as f:
    text_all = f.read()

In [4]:
# チャンクの分割
text_splitter = CharacterTextSplitter(
    separator='\n', # セパレート文字
    chunk_size=300, # チャンクのサイズ
    chunk_overlap=20, # オーバーラップの最大文字数
)
texts = text_splitter.split_text(text_all)

In [5]:
len(texts)

89

In [6]:
for text in texts:
    print(text[:10], ':', len(text))

皆さんおはようござい : 279
最も大切なページの一 : 293
知能の強さには2つの : 279
じゃあAIの強さとは : 277
一つはアルゴリズムの : 296
あら?
やばい
AG : 294
この
3文字の頭文字 : 288
ほぼ全部の分野で
抜 : 300
まだ知恵の方が上だと : 296
AGIの定義は
人類 : 300
仕事ってなんだと
会 : 291
それは僕の勝手な思い : 289
自分は一体どういう考 : 294
遅れてしまっているか : 290
自動車もぶつかって事 : 287
電気がどれほど有益で : 295
いくつか何回かこうや : 284
どのくらい優れている : 299
そこまでできるんです : 293
つまりもう疑問の余地 : 297
人類が生まれて
20 : 297
10年以内にですね
 : 281
GPTのですね
つい : 298
どこをどういじくれば : 292
いろんな会社が画像だ : 281
この生成AIはプログ : 294
自分で学習していくわ : 291
より重要なデータです : 296
例明記にですね
我々 : 297
世界の時価総額の
ト : 300
それに真正面から取り : 297
みんな広告を争ってい : 296
20年後に
一気に人 : 291
交通機関
モビリティ : 291
なんか無理やりこう
 : 291
コールセンターにおけ : 294
サポートするというの : 297
一言言えば
I wi : 296
遺伝子の解析によって : 300
つまり
好む好まない : 300
こういうような規制が : 291
一方
今日現在の
企 : 300
拒否するか
積極的に : 299
僕はソフトバンクを世 : 293
縦横斜めに
我々グル : 296
社員証はゼロです
す : 299
我々は
生成AI活用 : 297
おにぎり10個くらい : 291
僕の個人の
内に秘め : 300
戦略も今
打ち合わせ : 298
いうのがソフトバンク : 300
大きなLLMと
それ : 291
なんだチャットだろ
 : 297
このページ
AGIが : 299
10倍の次は1万倍だ : 299
その知能が
何倍ぐら : 293
つまり1万倍の差があ : 295
考える時が来たと
人 : 300
今会場で朝ですね
今 : 2

- セパレータ（デフォルトは'\n\n'）でテキストを小さなチャンクに分割
- 小さなチャンクを特定文字数になるまでマージし、大きなチャンクを作成

## ベクトルデーターベースの作成

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI, OpenAIChat

In [8]:
# ベクトルデータベースの作成
docsearch = FAISS.from_texts(
    embedding=OpenAIEmbeddings(), # 埋め込みモデル
    texts=texts # チャンク
)

In [9]:
# 質疑応答チェーンの作成
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAIChat(model='gpt-4-1106-preview', temperature=0), # LLM
    # llm=OpenAI(model='gpt-4-1106-preview', temperature=0), # LLM
    # chain_type='stuff', # チェーンタイプ
    chain_type='refine', # チェーンタイプ
    # chain_type='map_reduce', # チェーンタイプ
    # chain_type='map_rerank', # チェーンタイプ
    retriever=docsearch.as_retriever(), # リトリバー
)

/Users/ikedashinji/Desktop/corporate _collabo/langchain_239_env/lib/python3.10/site-packages/langchain/llms/openai.py:750: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [10]:
print(qa_chain.run('孫さんは何年にAGIができると言っていますか？日本語で答えてください。'))

新しいコンテキストを考慮に入れた上で、孫さんはAGI（人工汎用知能）が「10年以内に実現する」と述べており、AGIを使う人と使わない人の差は「人間対猿ぐらいの差ができる」とも指摘しています。この発言は、AGIの社会的影響の大きさを強調しているものであり、具体的な年数については「10年以内」という期間を示唆していますが、それ以上の詳細な年を明言しているわけではありません。したがって、元の回答に追加情報を加えて、以下のように答えることができます。

孫さんはAGIが「10年以内に実現する」と述べており、その社会的影響はAGIを使う人と使わない人の間で「人間対猿ぐらいの差ができる」と考えていますが、具体的な年を言っているわけではありません。


In [11]:
print(qa_chain.run('ASIとはなんですか？日本語で答えてください。'))

新しいコンテキストを踏まえて、お答えを修正いたします。

「ASI」とは「Artificial Superintelligence」の略で、「人工超知能」を意味します。これは、人間の知能をはるかに超える能力を持つ理論上のAIのことを指しています。一方で、コンテキストに出てきた「AGI」は「Artificial General Intelligence」、つまり「人工汎用知能」を意味し、あらゆる知的タスクで人間と同等に機能するAIを指します。現在のところは実現していない理論上の概念ですが、将来的にはAIが人間の知能を全ての分野で超える可能性があり、その時点で「人工超知能」、すなわちASIが現れることになります。

コンテキストによると、一般の人々の間では「AGI」についての認識がまだ低いようですが、その重要性を理解することは非常に価値があるとされています。AGIの実現は、人類の進化において大きな転換点となる可能性があり、それに続くASIの出現はさらに大きな影響をもたらすことでしょう。

したがって、お尋ねの「ASI」は「人工超知能」を指し、将来的には全ての分野で人間の知能を超えるAIという意味合いがあります。そして、現在議論されている「人工汎用知能」AGIの理解とその重要性を知ることは、我々にとって非常に重要であると言えるでしょう。


In [12]:
qa_chain.run('孫さんは生成AIを使ったことがない人になんと言っていますか？日本語で教えてください。')

'新しいコンテキストを考慮に入れた上で、元の回答を修正する必要があります。孫さんは、生成AIを使ったことがない人に対して、AIの可能性について語っているようです。彼はAIが特定の分野で人間の能力を超えていることを認めつつも、まだ全ての分野で人間を超えるには至っていないと述べています。そして、AIに限界があるという意見に対して、それはAIの限界ではなく、人々の理解に限界があると反論しています。彼は、将来的にAIがほぼ全ての分野で人間の知識を超えると信じていることを示唆しています。\n\nこのコンテキストに基づいて、孫さんが生成AIを使ったことがない人に言っていることは、AIの現在の能力と将来の可能性についての彼の見解であり、特に生成AIの活用コンテストについて言及しているわけではないようです。したがって、元の回答はこの新しいコンテキストには適していないため、以下のように修正します。\n\n孫さんは、生成AIを使ったことがない人に対して、AIが特定の分野では既に人間の能力を超えているが、全ての分野でそれが実現しているわけではないと説明しています。そして、AIに限界があるという意見に対しては、それはAIの限界ではなく、人々の理解に限界があると指摘しています。彼はAIの将来的な発展に楽観的であり、いずれAIがほぼ全ての分野で人間の知識を超えると考えているようです。'

### Retrieverのchainタイプの種類
- stuff  
すべてのデータをモデルに渡す。長文は機能しない。呼び出しは一回だけで済むので早い
- map_reduce  
チャンク毎にプロンプトを生成してLLMを呼び出す。大きなデータが使える。最後の結合時に一部の情報が失われてしまう。
- refine  
チャンク毎にプロンプトを生成するが、前のチャンクの出力と合わせてLLMを呼び出す。シーケンシャルな処理
- map_rerank  
チャンク毎にプロンプトを作ってLLMを呼び、応答をランク付けする。このスコアに基づいて応答する。ドキュメント間で情報が共有できない。一つの単純んな答えがあるときに最適。

  
### retrieverとは
コンピュータ用語では特定のキーワードや条件に基づいて、情報を収集してユーバーに情報を提供するシステムを表す。